In [1]:
!pip install opencv-python matplotlib tensorflow keras

In [2]:
import cv2
import numpy as np
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
# Load the trained model
model = load_model('asl_model.keras')

In [4]:
# Load class labels from folder names
dataset_path = 'dataset/asl_alphabet_train/'
class_names = sorted(os.listdir(dataset_path))  # A-Z, del, nothing, space
print("Class Labels:", class_names)

Class Labels: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [19]:
# Sentence Builder variables
sentence = ""
last_predicted_label = ""
frames_since_last_add = 0
delay_frames = 20 

In [20]:
# ImageDataGenerator to confirm classes (optional but useful for debugging)
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',
    subset='training'
)
print("Class Indices:", train_generator.class_indices)

Found 69600 images belonging to 29 classes.
Class Indices: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'del': 26, 'nothing': 27, 'space': 28}


In [21]:
# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Region of interest (ROI)
    x1, y1, x2, y2 = 100, 100, 300, 300
    roi = frame[y1:y2, x1:x2]

    # Preprocess ROI
    roi_resized = cv2.resize(roi, (64, 64))
    roi_gray = cv2.cvtColor(roi_resized, cv2.COLOR_BGR2GRAY)
    roi_normalized = roi_gray / 255.0
    roi_reshaped = np.reshape(roi_normalized, (1, 64, 64, 1))

    # Predict
    # Predict
    predictions = model.predict(roi_reshaped, verbose=0)
    confidence = np.max(predictions)
    class_id = np.argmax(predictions)

    # Get label from class_id
    predicted_label = class_names[class_id]

    # Show debug info in console (optional)
    print(f"Predicted: {predicted_label}, Confidence: {confidence:.2f}")

    # Filter by confidence threshold
    if confidence >= 0.75 and predicted_label != "nothing":
        if predicted_label == last_predicted_label:
            frames_since_last_add += 1
        else:
            frames_since_last_add = 1  # start fresh for new label

        # Add to sentence if label is stable for required frames
        if frames_since_last_add == delay_frames:
            if predicted_label == "space":
                sentence += " "
            elif predicted_label == "del":
                sentence = sentence[:-1]
            else:
                sentence += predicted_label

            frames_since_last_add = 0  # reset after adding
    else:
        frames_since_last_add = 0  # reset if not confident

    # Update last label
    last_predicted_label = predicted_label


    # Draw ROI box
    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

    # Show current prediction
    cv2.putText(frame, f"Predicted: {predicted_label}", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show full sentence
    cv2.putText(frame, f"Sentence: {sentence}", (10, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    # Show the frame
    cv2.imshow("Sign Language Detection", frame)

    # Quit or Clear
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('c'):
        sentence = ""

# Cleanup
cap.release()
cv2.destroyAllWindows()

Predicted: nothing, Confidence: 1.00
Predicted: nothing, Confidence: 1.00
Predicted: nothing, Confidence: 1.00
Predicted: nothing, Confidence: 1.00
Predicted: nothing, Confidence: 1.00
Predicted: nothing, Confidence: 1.00
Predicted: nothing, Confidence: 1.00
Predicted: nothing, Confidence: 1.00
Predicted: nothing, Confidence: 1.00
Predicted: nothing, Confidence: 1.00
Predicted: J, Confidence: 0.88
Predicted: J, Confidence: 0.91
Predicted: J, Confidence: 0.96
Predicted: J, Confidence: 0.96
Predicted: J, Confidence: 0.90
Predicted: J, Confidence: 0.80
Predicted: J, Confidence: 0.80
Predicted: J, Confidence: 0.70
Predicted: nothing, Confidence: 0.62
Predicted: nothing, Confidence: 0.92
Predicted: nothing, Confidence: 0.97
Predicted: nothing, Confidence: 0.70
Predicted: J, Confidence: 0.77
Predicted: J, Confidence: 0.90
Predicted: J, Confidence: 0.87
Predicted: J, Confidence: 0.80
Predicted: space, Confidence: 1.00
Predicted: space, Confidence: 1.00
Predicted: space, Confidence: 0.89
Predi